In [1]:
import pandas as pd
from IPython.display import display


In [2]:
pd.set_option('display.max_columns', None)  # show all columns
pd.set_option('display.width', 1000)        # set display width
pd.set_option('display.max_colwidth', 50)   # set max column width

In [3]:
import os
import pandas as pd

# === 1. Set folder paths ===
data_dir = "./data/source_datas"   # update to your own path, e.g., "./data/original/"
output_path = "./data/us_all_original_videos.csv"

# === 2. Files to merge ===
file_list = [
    "us_comedy.csv",
    "us_education.csv",
    "us_entertainment.csv",
    "us_gaming.csv",
    "us_howto_style.csv",
    "us_music.csv",
    "us_news_politics.csv",
    "us_science_technology.csv",
    "us_sports.csv",
    "us_travel_vlog.csv"
]

# === 3. Merge logic ===
dfs = []
for file in file_list:
    path = os.path.join(data_dir, file)
    df = pd.read_csv(path)

    # infer category name from filename (remove prefix "us_" and extension)
    category = (
        os.path.splitext(file)[0]
        .replace("us_", "")
        .replace(" 10.59.31PM", "")  # special case for that music file
    )
    df["category"] = category

    dfs.append(df)

# Concatenate into one table
merged_df = pd.concat(dfs, ignore_index=True)

# === 4. Clean / sort ===
# Deduplicate by video_id + upload_date
if set(["video_id", "upload_date"]).issubset(merged_df.columns):
    merged_df = merged_df.drop_duplicates(subset=["video_id", "upload_date"])

# Sort by category and upload date
if "upload_date" in merged_df.columns:
    merged_df = merged_df.sort_values(by=["category", "upload_date"])

# === 5. Export ===
merged_df.to_csv(output_path, index=False)
print(f"✅ Merge complete: {merged_df.shape[0]} rows saved to {output_path}")

✅ Merge complete: 4232 rows saved to ./data/us_all_original_videos.csv


In [4]:
import os
import glob
import pandas as pd

# === 1. Paths ===
followup_dir = "./data/Followups"         # folder containing follow-up CSVs
output_path  = "./data/followup_all.csv"  # output file path

# === 2. Load all follow-up files ===
followup_files = sorted(glob.glob(os.path.join(followup_dir, "followup_*.csv")))
print(f"📂 Found {len(followup_files)} follow-up files")

dfs = []

for f in followup_files:
    try:
        # extract date from filename
        date_str = os.path.basename(f).split("followup_")[-1].split(".csv")[0]
        date_str = date_str.replace("_", "-")

        # read CSV
        df = pd.read_csv(f)

        # standardize common column names
        df.rename(columns={
            "view_count": "views",
            "like_count": "likes",
            "comment_count": "comments"
        }, inplace=True)

        # add crawl date
        df["crawl_date"] = pd.to_datetime(date_str)

        dfs.append(df)
        print(f"✅ {os.path.basename(f)}: {df.shape[0]} rows")

    except Exception as e:
        print(f"⚠️ Failed to read {f}: {e}")

# === 3. Concatenate ===
followup_all = pd.concat(dfs, ignore_index=True)

# === 4. Deduplicate / sort ===
if {"video_id", "crawl_date"}.issubset(followup_all.columns):
    followup_all.drop_duplicates(subset=["video_id", "crawl_date"], inplace=True)
    followup_all.sort_values(by=["video_id", "crawl_date"], inplace=True)

followup_all.reset_index(drop=True, inplace=True)

print(f"📈 Merge complete: {followup_all.shape[0]} rows, {followup_all['video_id'].nunique()} videos")

# === 5. Export ===
followup_all.to_csv(output_path, index=False)
print(f"💾 Saved to: {output_path}")

📂 Found 55 follow-up files
✅ followup_2025-10-12.csv: 3901 rows
✅ followup_2025-10-13.csv: 3890 rows
✅ followup_2025-10-14.csv: 3876 rows
✅ followup_2025-10-15.csv: 3861 rows
✅ followup_2025-10-16.csv: 3852 rows
✅ followup_2025-10-17.csv: 3848 rows
✅ followup_2025-10-18.csv: 3835 rows
✅ followup_2025-10-19.csv: 3829 rows
✅ followup_2025-10-20.csv: 3823 rows
✅ followup_2025-10-21.csv: 3811 rows
✅ followup_2025-10-22.csv: 3799 rows
✅ followup_2025-10-23.csv: 3788 rows
✅ followup_2025-10-24.csv: 3786 rows
✅ followup_2025-10-25.csv: 3777 rows
✅ followup_2025-10-26.csv: 3745 rows
✅ followup_2025-10-27.csv: 3759 rows
✅ followup_2025-10-28.csv: 3755 rows
✅ followup_2025-10-29.csv: 3753 rows
✅ followup_2025-10-30.csv: 3753 rows
✅ followup_2025-10-31.csv: 3747 rows
✅ followup_2025-11-01.csv: 3745 rows
✅ followup_2025-11-02.csv: 3744 rows
✅ followup_2025-11-03.csv: 3740 rows
✅ followup_2025-11-04.csv: 3735 rows
✅ followup_2025-11-05.csv: 3732 rows
✅ followup_2025-11-06.csv: 3729 rows
✅ followup_

In [5]:
import pandas as pd

# === 1. Read master CSV ===
master_df = pd.read_csv("./data/us_all_original_videos.csv")

# === 2. followup_all already loaded ===
# Ensure video_id is str to avoid merge type mismatch
master_df["video_id"] = master_df["video_id"].astype(str)
followup_all["video_id"] = followup_all["video_id"].astype(str)

# === 3. Merge: one video_id with multiple followups (different crawl_date) ===
merged = master_df.merge(
    followup_all,
    on="video_id",
    how="left"               # keep all master rows
)

# === 4. Sort by video_id and date ===
if "crawl_date" in merged.columns:
    merged.sort_values(["video_id", "crawl_date"], inplace=True)

merged.reset_index(drop=True, inplace=True)

# === 5. Save ===
merged.to_csv("./master_with_followups.csv", index=False)
print("Merge done! Saved master_with_followups.csv")

Merge done! Saved master_with_followups.csv


In [6]:
#-------find the best popularity calculation function

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv("./data/master_with_followups.csv")

# Ensure dates are parsed
df["crawl_date"] = pd.to_datetime(df["crawl_date"])

# Sort by video_id and date
df = df.sort_values(["video_id", "crawl_date"])

# Find next crawled views per video
df["views_next"] = df.groupby("video_id")["views"].shift(-1)

# Keep necessary columns (including views_next and crawl_date)
df = df[["video_id", "views", "likes", "comments", "views_next", "crawl_date"]].dropna(subset=["video_id", "views", "likes", "comments"])
df.head()

,video_id,views,likes,comments,views_next,crawl_date
215129,--b_A-cZKK0,290.0,5.0,0.0,377.0,2025-10-12
215130,--b_A-cZKK0,377.0,6.0,0.0,402.0,2025-10-13
215131,--b_A-cZKK0,402.0,6.0,0.0,407.0,2025-10-14
215132,--b_A-cZKK0,407.0,6.0,0.0,407.0,2025-10-15
215133,--b_A-cZKK0,407.0,6.0,0.0,408.0,2025-10-16


In [10]:
df["log_views"] = np.log1p(df["views"])
df["log_likes"] = np.log1p(df["likes"])
df["log_comments"] = np.log1p(df["comments"])

df["like_rate"] = df["likes"] / df["views"].clip(lower=1)
df["comment_rate"] = df["comments"] / df["views"].clip(lower=1)

In [11]:
from sklearn.linear_model import LinearRegression

# Use log features to predict views_next (future views)
train_df = df.dropna(subset=["views_next"])  # drop samples without target
X = train_df[["log_views","log_likes","log_comments"]]
y = train_df["views_next"]  # target variable

reg = LinearRegression().fit(X, y)

weights = reg.coef_
intercept = reg.intercept_
print(f"Coefficients: {weights}")
print(f"Intercept: {intercept}")
print(f"Training samples: {len(train_df)}")

Coefficients: [260398.33409223  39871.85715901 164015.72649784]
Intercept: -2445124.35892001
Training samples: 218630


In [12]:
from sklearn.decomposition import PCA

# V1: original weighted formula
df["pop_V1"] = df["views"] + 5*df["likes"] + 10*df["comments"]

# V2: regression-based score predicting views_next (with intercept)
a, b, c = weights
df["pop_V2"] = a*df["log_views"] + b*df["log_likes"] + c*df["log_comments"] + intercept

# V3: ratio-based metric
df["pop_V3"] = df["like_rate"] + 3 * df["comment_rate"]

# V4: manual weighted log features (deduplicated)
df["pop_V4"] = df["log_views"] + 2*df["log_likes"] + 3*df["log_comments"]

# V5: PCA-derived component
pca = PCA(n_components=1)
df["pop_V5"] = pca.fit_transform(df[["log_views","log_likes","log_comments"]]).flatten()









In [13]:
df.columns

Index(['video_id', 'views', 'likes', 'comments', 'views_next', 'crawl_date', 'log_views', 'log_likes', 'log_comments', 'like_rate', 'comment_rate', 'pop_V1', 'pop_V2', 'pop_V3', 'pop_V4', 'pop_V5'], dtype='object')

In [14]:
import pandas as pd

df1 = pd.read_csv("./data/master_with_followups.csv")
df1["crawl_date"] = pd.to_datetime(df1["crawl_date"])

# Sort by video and crawl date
df1 = df1.sort_values(["video_id", "crawl_date"])

# First crawl date per video = day0
df1["day0"] = df1.groupby("video_id")["crawl_date"].transform("min")

# Compute relative day
df1["relative_day"] = (df1["crawl_date"] - df1["day0"]).dt.days

In [15]:
# Step 1: ensure dates are parsed (use df1, not df)
df1["crawl_date"] = pd.to_datetime(df1["crawl_date"])

# Step 2: pivot to get daily views table
pivot = df1.pivot_table(
    index="video_id",
    columns="relative_day",
    values="views",
    aggfunc="max"
)

# Step 3: cast columns to int (relative_day was float)
pivot.columns = pivot.columns.astype(int)

# Step 4: rename future views
pivot = pivot.rename(columns={
    1: "views_1d",
    7: "views_7d",
    30: "views_30d"
})

# Step 5: final views (latest)
pivot["views_final"] = pivot.max(axis=1)

# Keep only selected day columns
keep_cols = ["views_1d", "views_7d", "views_30d", "views_final"]
existing_cols = [c for c in keep_cols if c in pivot.columns]
pivot = pivot[existing_cols].reset_index()  # include video_id for merge

# Step 6: keep static info per video (title, tags, etc.)
static = df1.sort_values("crawl_date").groupby("video_id").first().reset_index()

# Step 7: merge
full_df = static.merge(pivot, on="video_id", how="left")

# Step 8: inspect columns
print(full_df.columns.tolist())

['video_id', 'id', 'title', 'description', 'hashtags', 'channel', 'published_at', 'category_id', 'duration', 'definition', 'category', 'views', 'likes', 'comments', 'crawl_date', 'day0', 'relative_day', 'views_1d', 'views_7d', 'views_30d', 'views_final']


In [16]:
full_df = df.merge(pivot, on="video_id", how="left")

In [17]:
print(full_df.columns.tolist())

['video_id', 'views', 'likes', 'comments', 'views_next', 'crawl_date', 'log_views', 'log_likes', 'log_comments', 'like_rate', 'comment_rate', 'pop_V1', 'pop_V2', 'pop_V3', 'pop_V4', 'pop_V5', 'views_1d', 'views_7d', 'views_30d', 'views_final']


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd

# Six popularity metrics to compare
pop_cols = ["pop_V1", "pop_V2", "pop_V3", "pop_V4", "pop_V5"]

# Four future view targets
targets = ["views_1d", "views_7d", "views_30d", "views_final"]

# Store results
results = {t: {} for t in targets}

# Drop rows with missing targets
clean_df = full_df.dropna(subset=targets)

for t in targets:
    for col in pop_cols:

        X = clean_df[[col]]
        y = clean_df[t]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=42
        )

        model = LinearRegression()
        model.fit(X_train, y_train)

        pred = model.predict(X_test)
        r2 = r2_score(y_test, pred)

        results[t][col] = r2

# Output results
print("\n=== R² scores for each popularity metric predicting future views ===\n")
df_results = pd.DataFrame(results)
print(df_results)

print("\nBest metric per target:")
print(df_results.idxmax())


=== R² scores for each popularity metric predicting future views ===

        views_1d  views_7d  views_30d  views_final
pop_V1  0.117492  0.876095   0.879876     0.879913
pop_V2  0.166561  0.044502   0.038056     0.040274
pop_V3  0.000384  0.000102   0.000084     0.000088
pop_V4  0.140282  0.037744   0.032563     0.034483
pop_V5  0.148216  0.040581   0.034775     0.036653

Best metric per target:
views_1d       pop_V2
views_7d       pop_V1
views_30d      pop_V1
views_final    pop_V1
dtype: object


In [19]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import numpy as np

pop_cols = ["pop_V1", "pop_V2", "pop_V3", "pop_V4", "pop_V5"]
targets = ["views_1d", "views_7d", "views_30d", "views_final"]

results_rf = {}

for t in targets:
    results_rf[t] = {}
    y = full_df[t]

    for col in pop_cols:
        X = full_df[[col]]

        # drop NA
        mask = ~X[col].isna() & ~y.isna()
        X = X[mask]
        y_clean = y[mask]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y_clean, test_size=0.1, random_state=42
        )

        model = RandomForestRegressor(
            n_estimators=300,
            max_depth=None,
            min_samples_split=4,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X_train, y_train)

        pred = model.predict(X_test)
        r2 = r2_score(y_test, pred)

        results_rf[t][col] = r2

# print output
df_rf = pd.DataFrame(results_rf).T
print(df_rf)

print("\nBest metric per target:")
print(df_rf.idxmax(axis=1))

               pop_V1    pop_V2    pop_V3    pop_V4    pop_V5
views_1d     0.791437  0.607073  0.034602  0.510597  0.689544
views_7d     0.964327  0.888906  0.471973  0.494143  0.836552
views_30d    0.933625  0.740689  0.383378  0.557234  0.790369
views_final  0.962399  0.844797  0.437349  0.713239  0.915944

Best metric per target:
views_1d       pop_V1
views_7d       pop_V1
views_30d      pop_V1
views_final    pop_V1
dtype: object


In [22]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import pandas as pd

pop_cols = ["pop_V1", "pop_V2", "pop_V3", "pop_V4", "pop_V5"]
targets   = ["views_1d", "views_7d", "views_30d", "views_final"]

results_lasso = pd.DataFrame(index=targets, columns=pop_cols)
results_ridge = pd.DataFrame(index=targets, columns=pop_cols)

df2 = full_df.copy()

# Remove rows where ALL future views are missing
df2 = df2.dropna(subset=targets, how="all")

for target in targets:
    for pop in pop_cols:
        df_tmp = df2[[pop, target]].dropna()
        
        X = df_tmp[[pop]].values
        y = df_tmp[target].values

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=42
        )

        # ---- LASSO ----
        lasso = Lasso(alpha=0.01)
        lasso.fit(X_train, y_train)
        pred_lasso = lasso.predict(X_test)
        r2_lasso = r2_score(y_test, pred_lasso)
        results_lasso.loc[target, pop] = r2_lasso

        # ---- RIDGE ----
        ridge = Ridge(alpha=1.0)
        ridge.fit(X_train, y_train)
        pred_ridge = ridge.predict(X_test)
        r2_ridge = r2_score(y_test, pred_ridge)
        results_ridge.loc[target, pop] = r2_ridge

print("\n=== LASSO R² Results ===")
display(results_lasso)

print("\nBest metric per target:")
print(df_rf.idxmax(axis=1))

print("\n=== RIDGE R² Results ===")
display(results_ridge)

print("\nBest metric per target:")
print(df_rf.idxmax(axis=1))

/Users/xuchen/myenv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.256e+15, tolerance: 2.212e+14
  model = cd_fast.enet_coordinate_descent(
/Users/xuchen/myenv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.482e+16, tolerance: 5.114e+14
  model = cd_fast.enet_coordinate_descent(



=== LASSO R² Results ===


/Users/xuchen/myenv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.551e+17, tolerance: 5.114e+14
  model = cd_fast.enet_coordinate_descent(


,pop_V1,pop_V2,pop_V3,pop_V4,pop_V5
views_1d,0.085641,0.166474,0.000196,0.140942,0.147921
views_7d,0.916679,0.04402,-0.000001,0.03791,0.040597
views_30d,0.80087,0.038459,-0.000009,0.033249,0.035387
views_final,0.963347,0.044226,-0.000078,0.038792,0.040769



Best metric per target:
views_1d       pop_V1
views_7d       pop_V1
views_30d      pop_V1
views_final    pop_V1
dtype: object

=== RIDGE R² Results ===


,pop_V1,pop_V2,pop_V3,pop_V4,pop_V5
views_1d,0.085641,0.166474,0.000196,0.140942,0.147921
views_7d,0.916679,0.04402,-0.000001,0.03791,0.040597
views_30d,0.80087,0.038459,-0.000009,0.033249,0.035387
views_final,0.963347,0.044226,-0.000078,0.038792,0.040769



Best metric per target:
views_1d       pop_V1
views_7d       pop_V1
views_30d      pop_V1
views_final    pop_V1
dtype: object


In [24]:
full_df.to_csv("master_with_scores.csv", index=False, encoding="utf-8")

In [28]:
import pandas as pd

df = pd.read_csv("master_with_followups.csv")


print("\n=== Column Names ===")
for c in df.columns:
    print(c)


=== Column Names ===
id
video_id
title
description
views_x
likes_x
comments_x
hashtags
channel
published_at
category_id
duration
definition
crawl_date_x
category
views_y
likes_y
comments_y
crawl_date_y


In [35]:
import pandas as pd

df = pd.read_csv("master_with_followups.csv")

df["video_id"] = df["video_id"].astype(str).str.strip()

meta_cols = ["title", "description", "category", "hashtags"]

df = df.sort_values(["video_id", "crawl_date"])

for col in meta_cols:
    df[col] = df.groupby("video_id")[col].transform(
        lambda x: [x.iloc[0]] + [""] * (len(x) - 1)
    )

df.to_csv("master_with_followups_cleaned.csv", index=False)

print("✅ master_with_followups cleaned & saved → master_with_followups_cleaned.csv")

✅ master_with_followups cleaned & saved → master_with_followups_cleaned.csv


In [47]:
import pandas as pd
import subprocess

file = "merged_results.csv"   # ← Change to your 4GB file name

# View column names
df_head = pd.read_csv(file, nrows=5)
print("Columns:", df_head.columns.tolist())

# Count rows without loading file
line_count = int(subprocess.check_output(["wc", "-l", file]).split()[0])
print("Total lines:", line_count)

Columns: ['id', 'video_id', 'title', 'description', 'hashtags', 'channel', 'published_at', 'category_id', 'duration', 'definition', 'category', 'views', 'likes', 'comments', 'crawl_date', 'views_next', 'crawl_date:1', 'log_views', 'log_likes', 'log_comments', 'like_rate', 'comment_rate', 'pop_V1', 'pop_V2', 'pop_V3', 'pop_V4', 'pop_V5', 'views_1d', 'views_7d', 'views_30d', 'views_final']
Total lines: 16358489


In [49]:
print("Column Count:", len(df_head.columns))
print(df_head.columns)

Column Count: 31
Index(['id', 'video_id', 'title', 'description', 'hashtags', 'channel', 'published_at', 'category_id', 'duration', 'definition', 'category', 'views', 'likes', 'comments', 'crawl_date', 'views_next', 'crawl_date:1', 'log_views', 'log_likes', 'log_comments', 'like_rate', 'comment_rate', 'pop_V1', 'pop_V2', 'pop_V3', 'pop_V4', 'pop_V5', 'views_1d', 'views_7d', 'views_30d', 'views_final'], dtype='object')


In [50]:
df_sample = pd.read_csv(file, nrows=100)
df_sample.describe(include="all")

,id,video_id,title,description,hashtags,channel,published_at,category_id,duration,definition,category,views,likes,comments,crawl_date,views_next,crawl_date:1,log_views,log_likes,log_comments,like_rate,comment_rate,pop_V1,pop_V2,pop_V3,pop_V4,pop_V5,views_1d,views_7d,views_30d,views_final
count,100.0,100,55,0.0,55,100,100,100.0,100,100,55,100.00,100.00,100.0,100,98.000000,100,100.000000,100.000000,100.0,100.000000,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.0
unique,NaN,1,1,NaN,1,1,1,NaN,1,1,1,NaN,NaN,NaN,2,NaN,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,--b_A-cZKK0,#bantayanislandcebu #travelvlog #travel #shor...,NaN,[],Allen Sille,2025-10-11T11:45:33Z,NaN,PT39S,hd,travel_vlog,NaN,NaN,NaN,2025-10-12,NaN,2025-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,100,55,NaN,55,100,100,NaN,100,100,55,NaN,NaN,NaN,55,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,345.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,329.15,5.45,0.0,NaN,407.204082,NaN,6.004937,1.942827,0.0,0.014787,0.0,434.760000,-803984.535543,0.014787,9.890592,-4.033926,377.0,408.0,408.0,408.0
std,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,43.50,0.50,0.0,NaN,4.467429,NaN,0.048919,0.021690,0.0,0.000392,0.0,17.754282,13581.609126,0.000392,0.091682,0.045702,0.0,0.0,0.0,0.0
min,345.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,290.00,5.00,0.0,NaN,377.000000,NaN,5.673323,1.791759,0.0,0.014706,0.0,315.000000,-896359.653761,0.014706,9.256842,-4.348471,377.0,408.0,408.0,408.0
25%,345.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,290.00,5.00,0.0,NaN,408.000000,NaN,6.013715,1.945910,0.0,0.014706,0.0,438.000000,-801575.899074,0.014706,9.905535,-4.026253,377.0,408.0,408.0,408.0
50%,345.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,290.00,5.00,0.0,NaN,408.000000,NaN,6.013715,1.945910,0.0,0.014706,0.0,438.000000,-801575.899074,0.014706,9.905535,-4.026253,377.0,408.0,408.0,408.0
75%,345.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0,NaN,NaN,NaN,377.00,6.00,0.0,NaN,408.000000,NaN,6.013715,1.945910,0.0,0.014706,0.0,438.000000,-801575.899074,0.014706,9.905535,-4.026253,377.0,408.0,408.0,408.0


In [52]:
import pandas as pd

df_iter = pd.read_csv(file, usecols=["video_id"], chunksize=200000)
total = 0

for chunk in df_iter:
    total += len(chunk)

print("True row count:", total)

True row count: 14093268


In [57]:
import pandas as pd

pd.options.mode.chained_assignment = None  # Disable any warning output

input_file = "merged_results.csv"
output_file = "CLEANED_MERGED.csv"

chunksize = 200_000

seen = set()
cleaned_chunks = 0

for chunk in pd.read_csv(input_file, chunksize=chunksize, low_memory=False):
    
    mask = []
    for vid, cdate in zip(chunk["video_id"], chunk["crawl_date"]):
        key = (vid, cdate)
        if key in seen:
            mask.append(False)
        else:
            seen.add(key)
            mask.append(True)

    cleaned_chunk = chunk[mask]

    if cleaned_chunks == 0:
        cleaned_chunk.to_csv(output_file, index=False, mode="w")
    else:
        cleaned_chunk.to_csv(output_file, index=False, mode="a", header=False)

    cleaned_chunks += 1